In [ ]:
import sys
sys.path.append("..")

In [ ]:
from remedi import data
data.disable_caching()
dataset = data.load_dataset("biosbias", split="train[5000:10000]")

In [ ]:
import json
from pathlib import Path

RESULTS_ROOT = Path("../results")
assert RESULTS_ROOT.exists()

In [ ]:
!ls ../results/icml_eval_bias_gen_gptj/linear/11

In [ ]:
EXPERIMENT_NAME = "icml_eval_bias_gen_gptj"
results_dir = RESULTS_ROOT / EXPERIMENT_NAME / "linear/11"

baseline_results_file = RESULTS_ROOT / EXPERIMENT_NAME / "baseline.json"
with baseline_results_file.open("r") as handle:
    baseline_results = json.load(handle)

editor_results_file = results_dir / "error_correction.json"
with editor_results_file.open("r") as handle:
    editor_results = json.load(handle)

In [ ]:
import random
# i = 21
# i = 33
# i = 52
# i = 55
# i = 252
# i = 299
# i = 1000
# i = 2153
# i = 3203
# i = 4102
# i = 727
# i = 3427
# i = 1039
i = random.choice(range(5000))
print((baseline_results["samples"][i]["prompt"] + baseline_results["samples"][i]["generation"]).replace("\n", " ").replace("  ", " "))
print()
print((baseline_results["samples"][i]["prompt"] + editor_results["samples"][i]["generation"]).replace("\n", " ").replace("  ", " "))

In [ ]:
i

In [ ]:
def clean_result(result):
    # Take just the first few sentences / the first thought.
    if "\n\n" in result:
        result = result.split("\n\n")[0]

    limit = 2 if "Inc. " not in result else 3
    result = ". ".join(result.split(". ")[:limit])

    # Sometimes CounterFact does not capitalize the entity, do so for presentation.
    result = result[0].upper() + result[1:]

    if not result.endswith("."):
        result += "."

    return result

rows = []
for index, corrects in (
#     21,
#     33,
    (52, (False, True)),
#     252,
#     299,
#     1000,
#     2153,
#     3203,
#     (4102, (False, True)), # Alternative to the below
    (55, (False, True)),
    
#     727,
#     3427,
    (1039, (False, True)),
    (3795, (True, True)),
    
):
    source = dataset[index]
    
    entity = source["entity"]

    attribute = source["context"]
    prompt = source["prompt"]

    prompt = r"\underline{\textbf{" + prompt + "}}"

    in_context = prompt + baseline_results["samples"][index]["generation"].replace(attribute, "")
    if index == 3795:
        in_context = in_context.replace("\n\n", " ")
    in_context = clean_result(in_context)
    edited = clean_result(prompt + editor_results["samples"][index]["generation"].replace(attribute, ""))

    in_context_marker = r"\correctmarker " if corrects[0] else r"\wrongmarker"
    edited_marker =  r"\correctmarker " if corrects[1] else f"\wrongmarker "
    
    in_context = in_context_marker + in_context
    edited = edited_marker + edited
    
    truncated_attribute = " ".join(attribute.split(" ")[:18])
    if truncated_attribute != attribute:
        truncated_attribute = truncated_attribute.rstrip() + r"\dots"

    if entity in ("Johanna", "Lauren"):
        truncated_attribute = truncated_attribute.replace(entity, entity + "'s")

    row = [
        index,
        entity,
        truncated_attribute,
        in_context,
        edited,
    ]
    rows.append(row)

rows

In [ ]:
table_str = (r"\\ " + "\n").join(
    " & ".join(r[1:]) for r in rows
) + r" \\"
print(table_str)